## Display the xml element tree

In [7]:
from pptx import Presentation

def display_shape_info(shape):
    """Display information about a shape."""
    if shape.has_text_frame:
        print(f"Text: {shape.text}")

    if shape.shape_type == 1:  # Auto Shape
        print("Type: Auto Shape")

    elif shape.shape_type == 13:  # Picture
        print("Type: Picture")
        print(f"Image Filename: {shape.image.filename}, Size: {shape.image.size}")

    elif shape.shape_type == 3:  # Table
        print("Type: Table")
        for row in shape.table.rows:
            for cell in row.cells:
                print(f"  Cell: {cell.text}")

    elif shape.shape_type == 19:  # Group Shape
        print("Type: Group Shape")

    elif shape.shape_type == 14:  # Line/Connector
        print("Type: Line/Connector")

    elif shape.shape_type == 18:  # Graphic Frame
        print("Type: Graphic Frame (Table/Chart/SmartArt)")

def display_pptx_shapes(pptx_path):
    """Display shapes in a PPTX file."""
    presentation = Presentation(pptx_path)
    for i, slide in enumerate(presentation.slides, start=1):
        print(f"\nSlide {i}:")
        for shape in slide.shapes:
            # display_shape_info(shape)
            print(shape)

# Replace with your actual PPTX file path
pptx_path = "/data/tianyuhu/PPTLayout/data/ppt/ZK7FNUZ33GBBCG7CFVYS56TQCTD72CJR.pptx"
display_pptx_shapes(pptx_path)



Slide 1:

Slide 2:

Slide 3:

Slide 4:

Slide 5:

Slide 6:

Slide 7:

Slide 8:

Slide 9:

Slide 10:

Slide 11:

Slide 12:

Slide 13:

Slide 14:
